In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [6]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [8]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


# Optimize model

In [9]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [10]:
mean_baseline = mean_squared_error(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
median_baseline = mean_absolute_error(y_df.values, np.repeat(np.median(y_df.values.squeeze()), y_df.shape[0]))
r2_baseline = r2_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
expl_var_baseline = explained_variance_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))

print(f"L2 baseline: {mean_baseline}")
print(f"L1 baseline: {median_baseline}")
print(f"R2 baseline: {r2_baseline}")
print(f"explained variance baseline: {expl_var_baseline}")

L2 baseline: 641687.6988062444
L1 baseline: 518.3333333333334
R2 baseline: 0.0
explained variance baseline: 0.0


In [11]:
def objective(h_params, X, y, loss_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingRegressor(
        # We use lad since it most closely matches up with hyper-parameter objective
        loss=loss_default,
        # Use this for the same reason
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [12]:
space = [
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * (len(space))
n_calls = 50 * (len(space))

In [13]:
loss_default = "ls"
scoring_default = "explained_variance"
callback_file = f"{unified_dsets[dset_idx]}_opt_gbr_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.07540275638507486, 533, 4, 'log2', 4, 1]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.2060
Function value obtained: 0.0641
Current minimum: 0.0641
Iteration No: 2 started. Evaluating function at random point.
[0.034738802219071975, 720, 5, 'log2', 4, 2]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.8461
Function value obtained: 0.0162
Current minimum: 0.0162
Iteration No: 3 started. Evaluating function at random point.
[0.08386822689969428, 343, 4, 'auto', 3, 3]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.1279
Function value obtained: 0.6375
Current minimum: 0.0162
Iteration No: 4 started. Evaluating function at random point.
[0.018853292027324983, 545, 3, 'log2', 3, 3]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.6286
Function value obtained: 0.0777
Current minimum: 0.0162
Iteration No: 5 started. Evaluating function at ra

KeyboardInterrupt: 

In [ ]:
loss_default = "lad"
scoring_default = "neg_mean_absolute_error"
callback_file = f"{unified_dsets[dset_idx]}_opt_gbr_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)